<a href="https://colab.research.google.com/github/repairedserver/Test/blob/master/Early_stopping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
cd/content/gdrive/My Drive/deeplearningbro/deeplearningbro/pytorch

/content/gdrive/My Drive/deeplearningbro/deeplearningbro/pytorch


In [8]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=transform)

trainset, valset = torch.utils.data.random_split(dataset, [30000, 20000])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=32, shuffle=False)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [12]:
class ResidualBlock (nn.Module):
  def __init__(self, in_channels, out_channels, stride=1):
    super (ResidualBlock, self).__init__()
    self.stride = stride
    self.in_channels= in_channels
    self.out_channels= out_channels
    self.conv_block = nn.Sequential(
        nn. Conv2d(self.in_channels, self.out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
        nn. BatchNorm2d (self.out_channels),
        nn.ReLU (),
        nn. Conv2d(self.out_channels, self.out_channels, kernel_size=3, stride=1, padding=1, bias=False),
        nn. BatchNorm2d(self.out_channels))
    
    if self.stride != 1 or self.in_channels != self.out_channels:
      self.downsample = nn.Sequential(
                      nn. Conv2d(self.in_channels, self.out_channels, kernel_size=1, stride=stride, bias=False),
                      nn. BatchNorm2d (self.out_channels))
            
  def forward(self,x):
    out= self.conv_block(x)
    if self.stride != 1 or self.in_channels != self.out_channels:
      x = self.downsample (x)
    out = F.relu(x + out)
    return out